In [5]:
import pandas as pd
import glob

In [6]:
files = glob.glob('*.xlsx')
files

['Water LevelSW101.5.xlsx',
 'Water LevelSW103.xlsx',
 'Water LevelSW219.xlsx',
 'Water LevelSW244.xlsx',
 'Water LevelSW28.xlsx']

In [7]:
def main(path):
    df = pd.read_excel(path, skiprows=9)
    

    df["DateTime"] = df["DATE TIME"]
    date = df["DateTime"].dt.date
    # time = df["DateTime"].dt.time
    df["Date"] = date
    df.drop(['DATE TIME', "DISTRICT", "SL", "UPAZILA", "RIVER", "STATION ID", \
            "STATION NAME", "DATA TYPE", "TYPE OF STATION", "LATITUDE", "LONGITUDE"],\
            axis=1, inplace=True) # delete the unnecessary columns


    # find if there is any missing date
    date_range = pd.date_range(df["Date"].min(), df["Date"].max())
    missing_dates = date_range[~date_range.isin(df["Date"])]
    df.drop("Date", axis=1, inplace=True)

    if len(missing_dates) > 0:
        print(f"There is missing dates in '{path}'. \nThe missing dates: {missing_dates}\n")
        # input(f"There is missing dates in {path}")
        return
    else:
        print(f"There is no missing dates in '{path}'")


    # delete the dupliate date
    # duplicates = df[df.duplicated(subset='DateTime', keep=False)]
    df = df.drop_duplicates(subset='DateTime', keep='first')

    # replace interval value with middle time
    i = 1
    while True:
        hours_diff_1 = (df.iloc[i, 1] - df.iloc[i-1, 1]).seconds / 3600
        hours_diff_2 = (df.iloc[i+1, 1] - df.iloc[i, 1]).seconds / 3600

        if hours_diff_1 % 3 != 0 and hours_diff_2 % 3 != 0:
            # print(df.iloc[i, 1])
        # if (df.iloc[i, 1] - df.iloc[i-1, 1]) != pd.Timedelta(hours=3) and (df.iloc[i+1, 1] - df.iloc[i, 1]) != pd.Timedelta(hours=3):
            # print(df.iloc[i, 1])
            # if (df.iloc[i, 1] - df.iloc[i-1, 1]) < pd.Timedelta(hours=1, minutes=30):
            #     df.iloc[i-1, 0] = df.iloc[i, 0]
            # else:
            #     df.iloc[i+1, 0] = df.iloc[i, 0]
            
            if (df.iloc[i, 1] - df.iloc[i-1, 1]) < (df.iloc[i+1, 1] - df.iloc[i, 1]):
                df.iloc[i-1, 0] = df.iloc[i, 0]
            else:
                df.iloc[i+1, 0] = df.iloc[i, 0]

        i += 1
        if i == len(df)-1:
            break

    # delete the row for that is not three hour interval
    df = df[df["DateTime"].dt.hour % 3 == 0]
    df = df[df["DateTime"].dt.minute == 0]

    # create 3 hour interval time that was not here
    df['DateTime'] = pd.to_datetime(df['DateTime'])
    df = df.set_index('DateTime')
    df = df.resample('3H').asfreq()

    df = df.reset_index()

    # fill the empty time with 12 hour after time
    df['WL (mMSL)'].fillna(df['WL (mMSL)'].shift(4), inplace=True)

    # save as csv
    df.to_csv(f"Corrected_{path[:-5]}.csv", index=False)
    print("Done\n")

In [8]:
for file in files:
    main(file)

There is no missing dates in 'Water LevelSW101.5.xlsx'
Done

There is no missing dates in 'Water LevelSW103.xlsx'
Done

There is missing dates in 'Water LevelSW219.xlsx'. 
The missing dates: DatetimeIndex(['2020-09-01', '2020-09-02', '2020-09-03', '2020-09-04',
               '2020-09-05', '2020-09-06', '2020-09-07', '2020-09-08',
               '2020-09-09', '2020-09-10', '2020-09-11', '2020-09-12',
               '2020-09-13', '2020-09-14', '2020-09-15', '2020-09-16',
               '2020-09-17', '2020-09-18', '2020-09-19', '2020-09-20',
               '2020-09-21', '2020-09-22', '2020-09-23', '2020-09-24',
               '2020-09-25', '2020-09-26', '2020-09-27', '2020-09-28',
               '2020-09-29', '2020-09-30'],
              dtype='datetime64[ns]', freq='D')

There is no missing dates in 'Water LevelSW244.xlsx'
Done

There is no missing dates in 'Water LevelSW28.xlsx'
Done

